In [2]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

import pickle

from functions import *
from data_cleaning import DataCleaning

from betfairlightweight import StreamListener, APIClient
import os


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/Users/andrewbarwise/miniforge3/envs/horse_trading/lib/python3.11/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/andrewbarwise/miniforge3/envs/horse_trading/lib/python3.11/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [3]:
df = pd.read_csv('../data/raw_proform/jumps_data.csv')

df.head(5)

,Race Time,Course,Distance (y),Race Type,Official Going,Classifications,Draw,Draw IV,Position,Horse,Jockey Claim,Weight (pounds),DSLR,Dam,Sire,Form String,SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Decimal,PFR Rank,Main,Main Rank,Main Stats,Stats,Stats Rank,Trn Stats,Jky Stats,TrnJky Stats,Hrs Stats,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,4th LTOt Speed Rating,Evening Price,Breakfast Price,Morning Price,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Finished Races SR,WON SR Before,Won P/L Before,Plc SR Before,Distance LTO (Same Code),Distance In Yards LTO (Same Code),Pounds LTO (Same Code),Official Rating LTO (Same Code),Distance To Winner LTO (Same Code),DSLR(F),DSLR(J),FLAT TURF Runs Before,FLAT AW Runs Before,CHASE Runs Before,HURDLE Runs Before,NHF Runs Before,ALL FLAT Runs Before,FLAT TURF Wins Before,FLAT AW Wins Before,CHASE Wins Before,HURDLE Wins Before,NHF Wins Before,Distance In Yards LTO,Official Rating LTO,Distance To Winner LTO,Betfair SP Odds LTO (Decimcal),Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,JSH Mean Movement Rank,Cherry Picked Rating,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Course Wins,Today's Course Places,Today's Course Runs,Heavy/Soft Ground Wins,Heavy/Soft Ground Places,Heavy/Soft Ground Runs,Good to Soft Ground Wins,Good to Soft Ground Runs,Good Ground Wins,Good Ground Runs,Good to Firm Ground Wins,Good to Firm Ground Runs,SpeedRating LTO
0,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,1.0,Attacca,0,154,221.0,Listening,Mahler,2,1.9,2.03,1,0,0,0,0,2,3,0,1,5.00,2,328,3,690,362,1,94,99,97,72,0,3,0,3,0,3,0,3,0,3,0,3,0,2,0,0,2.75,2.63,2.2,57.0,57.0,57.0,57.0,57.0,57.0,NaN,NaN,0.00,-1.00,100.00,2m,3520,148.0,0.0,2.75,NaN,221.0,0,0,0,0,1,0,0,0,0,0,0,3520,0.0,2.75,6.86,57.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,57.0
1,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,2.0,Alien Storm,0,161,21.0,Missusan,Getaway,5321,4.5,6.40,0,88,88,0,0,0,4,0,1,2.25,1,360,1,675,315,5,81,75,83,76,63,1,63,1,63,1,63,1,63,1,63,1,0,2,0,0,3.75,3.75,4.5,65.0,63.0,57.0,55.0,60.0,58.0,63.0,100.0,25.00,1.00,0.00,1m 7f 195y,3495,153.0,0.0,0.00,NaN,21.0,0,0,0,1,3,0,0,0,0,1,0,3495,0.0,0.00,5.70,63.0,0.0,0.0,0.0,0.0,NaN,NaN,1,0,2,1,0,2,1,0,1,0,0,1,0,0,0,1,2,0,2,0,0,63.0
2,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,3.0,Celtic Art,0,154,35.0,Irish Song,Mastercraftsman,232-1313690,17.0,21.30,0,0,0,0,0,6,1,0,1,NaN,5,322,4,653,331,4,78,94,79,80,0,3,0,3,0,3,0,3,0,3,0,3,0,2,0,0,10.00,12.00,19.0,73.0,64.0,62.0,61.0,64.0,64.0,71.0,NaN,13.64,-10.92,27.27,NaN,0,NaN,NaN,NaN,35.0,NaN,16,6,0,0,0,22,1,2,0,0,0,2519,88.0,9.65,93.46,64.0,70.0,70.0,53.0,88.0,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,7,1,8,64.0
3,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,4.0,Tellmesomethingood,0,154,NaN,Norabelle,Walk In The Park,NaN,12.0,13.70,0,0,0,0,0,0,4,0,1,NaN,6,285,7,631,346,3,97,82,87,80,0,3,0,3,0,3,0,3,0,3,0,3,0,2,0,0,13.00,11.00,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
4,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,5.0,Iskar D'Airy,0,154,12.0,Canzka D'Airy,Sinndar,7349,81.0,160.00,0,46,40,52,0,0,4,0,1,19.00,

In [4]:
df['odds_rank'] = (
    df.groupby(['Race Time', 'Course'])['BF Decimal SP']
    .rank(method='min', ascending=True)
)

df['morning_price_rank'] = (
    df.groupby(['Race Time', 'Course'])['Morning Price']
    .rank(method='min', ascending=True)
)

df[['Race Time','BF Decimal SP', 'odds_rank', 'morning_price_rank']].head(10)

,Race Time,BF Decimal SP,odds_rank,morning_price_rank
0,07/11/2022 12:50:00 PM,2.03,1.0,1.0
1,07/11/2022 12:50:00 PM,6.40,3.0,3.0
2,07/11/2022 12:50:00 PM,21.30,5.0,5.0
3,07/11/2022 12:50:00 PM,13.70,4.0,4.0
4,07/11/2022 12:50:00 PM,160.00,7.0,6.0
5,07/11/2022 12:50:00 PM,4.83,2.0,2.0
6,07/11/2022 12:50:00 PM,150.00,6.0,7.0
7,07/11/2022 1:25:00 PM,4.84,2.0,2.0
8,07/11/2022 1:25:00 PM,4.62,1.0,1.0
9,07/11/2022 1:25:00 PM,35.20,7.0,7.0


In [5]:
winners = df[df['Won (1=Won, 0=Lost)'] == 1]
winners.head()

,Race Time,Course,Distance (y),Race Type,Official Going,Classifications,Draw,Draw IV,Position,Horse,Jockey Claim,Weight (pounds),DSLR,Dam,Sire,Form String,SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Decimal,PFR Rank,Main,Main Rank,Main Stats,Stats,Stats Rank,Trn Stats,Jky Stats,TrnJky Stats,Hrs Stats,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,4th LTOt Speed Rating,Evening Price,Breakfast Price,Morning Price,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Finished Races SR,WON SR Before,Won P/L Before,Plc SR Before,Distance LTO (Same Code),Distance In Yards LTO (Same Code),Pounds LTO (Same Code),Official Rating LTO (Same Code),Distance To Winner LTO (Same Code),DSLR(F),DSLR(J),FLAT TURF Runs Before,FLAT AW Runs Before,CHASE Runs Before,HURDLE Runs Before,NHF Runs Before,ALL FLAT Runs Before,FLAT TURF Wins Before,FLAT AW Wins Before,CHASE Wins Before,HURDLE Wins Before,NHF Wins Before,Distance In Yards LTO,Official Rating LTO,Distance To Winner LTO,Betfair SP Odds LTO (Decimcal),Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,JSH Mean Movement Rank,Cherry Picked Rating,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Course Wins,Today's Course Places,Today's Course Runs,Heavy/Soft Ground Wins,Heavy/Soft Ground Places,Heavy/Soft Ground Runs,Good to Soft Ground Wins,Good to Soft Ground Runs,Good Ground Wins,Good Ground Runs,Good to Firm Ground Wins,Good to Firm Ground Runs,SpeedRating LTO,odds_rank,morning_price_rank
0,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,1.0,Attacca,0,154,221.0,Listening,Mahler,2,1.90,2.03,1,0,0,0,0,2,3,0,1,5.00,2,328,3,690,362,1,94,99,97,72,0,3,0,3,0,3,0,3,0,3,0,3,0,2,0,0,2.75,2.63,2.2,57.0,57.0,57.0,57.0,57.0,57.0,NaN,NaN,0.00,-1.00,100.00,2m,3520,148.0,0.0,2.75,NaN,221.0,0,0,0,0,1,0,0,0,0,0,0,3520,0.0,2.75,6.86,57.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,57.0,1.0,1.0
7,07/11/2022 1:25:00 PM,Kempton,4510,Chase,Good To Soft,Hcap-Nov-Cha,0,NaN,1.0,Dalamoi,0,165,200.0,Dalamine,Pour Moi,2512-35,4.50,4.84,1,0,0,0,0,4,5,117,1,7.50,8,299,5,574,275,8,77,72,76,50,0,3,0,2,0,3,0,2,0,2,0,2,0,1,0,0,8.00,7.00,6.0,76.0,64.0,60.0,61.0,64.0,64.0,68.0,100.0,16.67,-3.13,50.00,2m 3f,4180,166.0,119.0,15.80,NaN,200.0,0,0,0,4,2,0,0,0,0,1,0,4180,119.0,15.80,4.78,64.0,86.0,78.0,85.0,95.0,NaN,NaN,0,0,0,0,1,1,1,2,4,0,0,0,1,2,3,0,0,0,3,0,0,64.0,2.0,2.0
15,07/11/2022 2:00:00 PM,Kempton,4620,Hurdle,Good To Soft,Nov-Hdle,0,NaN,1.0,Twig,7,147,16.0,Southern Exit,Sulamani,5/7-11212,1.25,1.30,1,89,89,0,0,6,2,132,1,2.75,1,366,1,716,350,4,89,93,85,83,79,1,79,1,79,1,79,1,79,1,79,1,0,1,0,0,1.50,1.40,1.4,79.0,79.0,59.0,71.0,60.0,70.0,70.0,100.0,42.86,-0.27,14.29,2m 7f 208y,5268,154.0,134.0,1.50,NaN,16.0,0,0,4,1,2,0,0,0,3,0,0,5268,134.0,1.50,13.50,79.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,3,5,0,0,79.0,1.0,1.0
21,07/11/2022 2:35:00 PM,Kempton,3960,Chase,Good To Soft,Hcap-Cha,0,NaN,1.0,Frere D'Armes,0,158,219.0,Ville Sainte,Bathyrhon,2-1165,2.88,2.88,1,0,0,0,0,8,3,120,6,8.00,5,281,4,615,334,1,91,94,93,56,0,9,0,9,0,9,0,9,0,9,0,9,0,9,0,0,5.00,5.00,4.5,73.0,57.0,63.0,64.0,62.0,62.0,68.0,100.0,40.00,-1.98,20.00,2m 2f 183y,4143,158.0,122.0,32.75,NaN,219.0,0,0,0,5,0,0,0,0,0,2,0,4143,122.0,32.75,8.40,57.0,91.0,85.0,96.0,93.0,NaN,NaN,1,1,3,0,0,0,1,0,1,0,0,1,1,0,1,1,3,0,1,0,0,57.0,1.0,2.0
31,07/11/2022 3:05:00 PM,Kempton,4620,Hurdle,Good To

In [6]:
def calculate_proportions(df, column_name):
    proportions = df[column_name].value_counts(normalize=True)
    return proportions


In [7]:
def calculate_grouped_proportions(df, group_columns):
    df['proportion'] = df.groupby(group_columns)[group_columns[0]].transform('count') / len(df)
    return df

In [8]:
main_rank = calculate_proportions(winners, 'Main Rank')
main_rank.head()

Main Rank
1    0.257342
2    0.180289
3    0.135490
4    0.102638
5    0.082728
Name: proportion, dtype: float64

In [9]:
mr_career_spedd_rank = calculate_proportions(winners, 'MR Career Speed Rating Rank')
mr_career_spedd_rank.head()

MR Career Speed Rating Rank
1    0.230861
2    0.164360
3    0.128522
4    0.118069
5    0.088203
Name: proportion, dtype: float64

In [10]:
lto_speed_rank = calculate_proportions(winners, 'LTO Speed Rating Rank')
lto_speed_rank.head()

LTO Speed Rating Rank
1    0.231160
2    0.176108
3    0.137680
4    0.115580
5    0.092583
Name: proportion, dtype: float64

In [11]:
odds_rank_props = calculate_proportions(winners, 'odds_rank')
odds_rank_props.head()

odds_rank
1.0    0.362270
2.0    0.206869
3.0    0.136685
4.0    0.100846
5.0    0.064510
Name: proportion, dtype: float64

In [12]:
pace_rating_props = calculate_proportions(winners, 'Pace Rating Rank')
pace_rating_props.head()

Pace Rating Rank
1    0.235540
2    0.177402
3    0.145744
4    0.122150
5    0.092484
Name: proportion, dtype: float64

In [13]:
or_rating_props = calculate_proportions(winners, 'OR Rank')
or_rating_props.head()

OR Rank
1    0.336884
2    0.170134
3    0.124938
4    0.090791
5    0.073171
Name: proportion, dtype: float64

In [14]:
len(winners)

10045

In [15]:
df['boolean'] = (df['morning_price_rank'] <=3) & (df['OR Rank'] <=3) & (df['Main Rank'] <=3) & (df['Pace Rating Rank'] <=3) & (df['TrnJky Stats'] > 80) & (df["Today's Going Wins"] > 1)


df.head()

,Race Time,Course,Distance (y),Race Type,Official Going,Classifications,Draw,Draw IV,Position,Horse,Jockey Claim,Weight (pounds),DSLR,Dam,Sire,Form String,SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Decimal,PFR Rank,Main,Main Rank,Main Stats,Stats,Stats Rank,Trn Stats,Jky Stats,TrnJky Stats,Hrs Stats,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,4th LTOt Speed Rating,Evening Price,Breakfast Price,Morning Price,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Finished Races SR,WON SR Before,Won P/L Before,Plc SR Before,Distance LTO (Same Code),Distance In Yards LTO (Same Code),Pounds LTO (Same Code),Official Rating LTO (Same Code),Distance To Winner LTO (Same Code),DSLR(F),DSLR(J),FLAT TURF Runs Before,FLAT AW Runs Before,CHASE Runs Before,HURDLE Runs Before,NHF Runs Before,ALL FLAT Runs Before,FLAT TURF Wins Before,FLAT AW Wins Before,CHASE Wins Before,HURDLE Wins Before,NHF Wins Before,Distance In Yards LTO,Official Rating LTO,Distance To Winner LTO,Betfair SP Odds LTO (Decimcal),Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,JSH Mean Movement Rank,Cherry Picked Rating,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Course Wins,Today's Course Places,Today's Course Runs,Heavy/Soft Ground Wins,Heavy/Soft Ground Places,Heavy/Soft Ground Runs,Good to Soft Ground Wins,Good to Soft Ground Runs,Good Ground Wins,Good Ground Runs,Good to Firm Ground Wins,Good to Firm Ground Runs,SpeedRating LTO,odds_rank,morning_price_rank,boolean
0,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,1.0,Attacca,0,154,221.0,Listening,Mahler,2,1.9,2.03,1,0,0,0,0,2,3,0,1,5.00,2,328,3,690,362,1,94,99,97,72,0,3,0,3,0,3,0,3,0,3,0,3,0,2,0,0,2.75,2.63,2.2,57.0,57.0,57.0,57.0,57.0,57.0,NaN,NaN,0.00,-1.00,100.00,2m,3520,148.0,0.0,2.75,NaN,221.0,0,0,0,0,1,0,0,0,0,0,0,3520,0.0,2.75,6.86,57.0,0.0,0.0,0.0,0.0,NaN,NaN,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,57.0,1.0,1.0,False
1,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,2.0,Alien Storm,0,161,21.0,Missusan,Getaway,5321,4.5,6.40,0,88,88,0,0,0,4,0,1,2.25,1,360,1,675,315,5,81,75,83,76,63,1,63,1,63,1,63,1,63,1,63,1,0,2,0,0,3.75,3.75,4.5,65.0,63.0,57.0,55.0,60.0,58.0,63.0,100.0,25.00,1.00,0.00,1m 7f 195y,3495,153.0,0.0,0.00,NaN,21.0,0,0,0,1,3,0,0,0,0,1,0,3495,0.0,0.00,5.70,63.0,0.0,0.0,0.0,0.0,NaN,NaN,1,0,2,1,0,2,1,0,1,0,0,1,0,0,0,1,2,0,2,0,0,63.0,3.0,3.0,False
2,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,3.0,Celtic Art,0,154,35.0,Irish Song,Mastercraftsman,232-1313690,17.0,21.30,0,0,0,0,0,6,1,0,1,NaN,5,322,4,653,331,4,78,94,79,80,0,3,0,3,0,3,0,3,0,3,0,3,0,2,0,0,10.00,12.00,19.0,73.0,64.0,62.0,61.0,64.0,64.0,71.0,NaN,13.64,-10.92,27.27,NaN,0,NaN,NaN,NaN,35.0,NaN,16,6,0,0,0,22,1,2,0,0,0,2519,88.0,9.65,93.46,64.0,70.0,70.0,53.0,88.0,NaN,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,7,1,8,64.0,5.0,5.0,False
3,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,4.0,Tellmesomethingood,0,154,NaN,Norabelle,Walk In The Park,NaN,12.0,13.70,0,0,0,0,0,0,4,0,1,NaN,6,285,7,631,346,3,97,82,87,80,0,3,0,3,0,3,0,3,0,3,0,3,0,2,0,0,13.00,11.00,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,4.0,4.0,False
4,07/11/2022 12:50:00 PM,Kempton,3520,Hurdle,Good To Soft,Nov-Hdle,0,NaN,5

In [16]:
df['matching'] = df['Won (1=Won, 0=Lost)'] == df['boolean'].astype(int)

In [17]:
boolean_prop = calculate_proportions(df, 'boolean')
boolean_prop

boolean
False    0.983402
True     0.016598
Name: proportion, dtype: float64

In [18]:
def profit_calculation(df, stake = 20):
    """
    Calculate ROI, total returns from the model predictions.

    Args:
        df (pd.Dataframe): A dataframe that has a column holding the model predictions
        stake (int): The amount staked for each bet

    Returns:
        print() 
    """
    # Filter rows where model_preds == 1
    bets = df[df['boolean'] == True].copy()

    # Calculate returns
    bets['Return'] = bets.apply(
        lambda row: (row['BF Decimal SP'] - 1) * stake if row['Won (1=Won, 0=Lost)'] == 1 else -1,
        axis=1
    )

    # Total return
    total_return = bets['Return'].sum()

    # total number of bets
    total_bets = len(bets)

    # Calculate accuracy: Percentage of correct predictions where the model predicted 1 and won
    correct_predictions = bets[bets['Won (1=Won, 0=Lost)'] == 1].shape[0]

    if total_bets > 0:
        accuracy = (correct_predictions / total_bets) * 100  # Accuracy in percentage
        return_per_pound = total_return / (total_bets * stake)
    else:
        accuracy = 0
        return_per_pound = 0

    print(f"Total number of bets: {total_bets}")
    print(f"Total number of correct predictions: {correct_predictions}")
    print(f"Total return from betting £{stake:.2f} on each prediction where model_preds == 1: £{total_return:.2f}")
    print(f"Return per pound invested: £{return_per_pound:.2f}")
    print(f"Model accuracy: {accuracy:.2f}%")

In [19]:
profit_calculation(df)

Total number of bets: 1559
Total number of correct predictions: 488
Total return from betting £20.00 on each prediction where model_preds == 1: £22833.80
Return per pound invested: £0.73
Model accuracy: 31.30%


In [25]:
df1 = pd.read_csv('../data/raw_proform/2024_11_14.csv')

df1['morning_price_rank'] = (
    df1.groupby(['Race Time', 'Course'])['Morning Price']
    .rank(method='min', ascending=True)
)

df1['boolean'] = (df1['morning_price_rank'] <=3) & (df1['OR Rank'] <=3) & (df1['Main Rank'] <=3) & (df1['Pace Rating Rank'] <=3) & (df1['TrnJky Stats'] > 80) & (df1["Today's Going Wins"] > 1)

In [26]:
aa = df1[df1['boolean']== True]
aa

,Race Time,Course,Distance (y),Race Type,Official Going,Classifications,Draw,Draw IV,Position,Horse,Jockey Claim,Weight (pounds),DSLR,Dam,Sire,Form String,SP Odds Decimal,BF Decimal SP,"Won (1=Won, 0=Lost)",PRC Average,PRC Last Run,PRC 2nd Last Run,PRC 3rd Last Run,Pace Rating,Pace Rating Rank,OR,OR Rank,Tissue Odds Decimal,PFR Rank,Main,Main Rank,Main Stats,Stats,Stats Rank,Trn Stats,Jky Stats,TrnJky Stats,Hrs Stats,HA Career Speed Rating,HA Career Speed Rating Rank,HA Last 1 Year Speed Rating,HA Last 1 Year Speed Rating Rank,MR Career Speed Rating,MR Career Speed Rating Rank,MR Last 1 Year Speed Rating,MR Last 1 Year Speed Rating Rank,MR Last 3 Runs Speed Rating,MR Last 3 Runs Speed Rating Rank,LTO Speed Rating,LTO Speed Rating Rank,2nd LTO Speed Rating,2nd LTO Speed Rating Rank,3rd LTO Speed Rating,4th LTOt Speed Rating,Evening Price,Breakfast Price,Morning Price,dyHA,dyLTO,dyAVG,dyAVG(x),dyMR,dyMR(x),dy2HA,Finished Races SR,WON SR Before,Won P/L Before,Plc SR Before,Distance LTO (Same Code),Distance In Yards LTO (Same Code),Pounds LTO (Same Code),Official Rating LTO (Same Code),Distance To Winner LTO (Same Code),DSLR(F),DSLR(J),FLAT TURF Runs Before,FLAT AW Runs Before,CHASE Runs Before,HURDLE Runs Before,NHF Runs Before,ALL FLAT Runs Before,FLAT TURF Wins Before,FLAT AW Wins Before,CHASE Wins Before,HURDLE Wins Before,NHF Wins Before,Distance In Yards LTO,Official Rating LTO,Distance To Winner LTO,Betfair SP Odds LTO (Decimcal),Speed Rating LTO,PRC Average LTO,PRC Last Run LTO,PRC 2nd Last Run LTO,PRC 3rd Last Run LTO,JSH Mean Movement Rank,Cherry Picked Rating,Today's Going Wins,Today's Going Places,Today's Going Runs,Today's Distance Wins,Today's Distance Places,Today's Distance Runs,Today's Class Wins,Today's Class Places,Today's Class Runs,Today's Course Wins,Today's Course Places,Today's Course Runs,Heavy/Soft Ground Wins,Heavy/Soft Ground Places,Heavy/Soft Ground Runs,Good to Soft Ground Wins,Good to Soft Ground Runs,Good Ground Wins,Good Ground Runs,Good to Firm Ground Wins,Good to Firm Ground Runs,SpeedRating LTO,morning_price_rank,boolean
59,14/11/2024 1:30:00 PM,Market Rasen,5251,Chase,Good,Hcap-Nov-Cha,0,NaN,NaN,Moon Hunter,0,167,34.0,Miss Duffy,Pether's Moon,11/P2048-5F4,NaN,1,0,62,79,53,53,2,3,119,2,8.5,3,329,3,678,349,4,95,94,92,68,73,1,65,1,63,1,62,2,62,2,65,1,45,2,62,0,4.33,2.5,2.88,74.0,65.0,51.0,57.0,62.0,45.0,73.0,80.0,20.0,0.0,10.0,2m 7f 131y,5191,150.0,120.0,13.75,NaN,34.0,0,0,5,5,0,0,0,0,0,2,0,5191,120.0,13.75,13.76,65.0,55.0,53.0,53.0,58.0,NaN,NaN,2,1,4,0,0,4,1,1,2,0,0,1,0,0,1,0,5,2,4,0,0,65.0,3.0,True


In [27]:
aa[['Race Time', 'Course', 'Horse']]

,Race Time,Course,Horse
59,14/11/2024 1:30:00 PM,Market Rasen,Moon Hunter
